In [ ]:
import numpy as np
from skimage import feature, color
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

from keras.datasets import cifar10

In [ ]:
# Carrega o dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
# calcula lbp
def LBP(X, lbp_features, color_features):
    for image in X:
        # lbp
        gray_image = color.rgb2gray(image)  # transforma em preto e branco
        lbp = feature.local_binary_pattern(gray_image, n_points, radius, method="uniform")
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        lbp_hist = lbp_hist / (lbp_hist.sum() + 1e-6)  # normalizacao
        lbp_features.append(lbp_hist) # coloca na lista de features

        # cor utilizando a media e o desvio padrao
        color_stats = np.hstack([np.mean(image, axis=(0, 1)), np.std(image, axis=(0, 1))])
        color_features.append(color_stats)

In [ ]:
# parametros do lbp
radius = 1
n_points = 8 * radius

lbp_features_train = []
color_features_train = []
LBP(X_train, lbp_features_train, color_features_train)

lbp_features_test = []
color_features_test = []
LBP(X_test, lbp_features_test, color_features_test)

In [ ]:
# Concatena os dois vetores
def feature_concat(lbp_features, color_features):
    concatenated_features = []
    for lbp_feats, color_feats in zip(lbp_features, color_features):
        concatenated_features.append(np.concatenate((lbp_feats, color_feats)))
    return concatenated_features


features_train = feature_concat(lbp_features_train, color_features_train)
features_test = feature_concat(lbp_features_test, color_features_test)

In [ ]:
# faz split para ter uma parte de validacao tambem
X_train, X_val, y_train, y_val = train_test_split(features_train, y_train, test_size=0.2, random_state=42)

In [ ]:
# treino
clf = RandomForestClassifier(n_estimators=300, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=0)

In [ ]:
# validacao
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("acurácia da validação:", accuracy)

acurácia da validação: 0.39


In [ ]:
# teste
y_pred = clf.predict(features_test)
accuracy = accuracy_score(y_test, y_pred)
print("acurácia do teste:", accuracy)

acurácia do teste: 0.389


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [ ]:
import pandas as pd
results_dict = { "Model": 'LBP', "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1 Score": f1}

pd.DataFrame.from_dict([results_dict])

,Model,Accuracy,Precision,Recall,F1 Score
0,LBP,0.389,0.385826,0.389,0.382735
